In [ ]:
import os
os.getcwd()
os.chdir("..")

In [ ]:
os.getcwd()


In [ ]:
# Optimizing the chucking process
import os
import pdfplumber
import re


# python PDFProcessing/PDFProccesing.py > outputs/refoutput.txt

text_list = []

with pdfplumber.open("PDF_docs/doc_0.pdf") as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        if text:
            text_list.append(text)
            text = text.join(text_list)
text





In [ ]:
parts = re.split(r"\breferences\b", text, flags=re.IGNORECASE, maxsplit=1)

body = parts[0].strip()  # Everything before "References"
reference = parts[1].strip() if len(parts) > 1 else ""  # Everything after "References"



In [ ]:
reference

In [ ]:
# Get meta data

In [ ]:
pdf = pdfplumber.open("PDF_docs/doc_2.pdf")
pdf.metadata

In [ ]:
pdf = pdfplumber.open("PDF_docs/doc_0.pdf")
metadata = pdf.metadata  # Extrahiere Metadaten
if metadata and "Title" in metadata and metadata["Title"]:
    print( f"Titel: {metadata['Title']}")
else: 
    print(False)

In [ ]:
with pdfplumber.open("PDF_docs/doc_1.pdf") as pdf:
    metadata = pdf.metadata
    print(metadata)

In [1]:
from ollama import ChatResponse, chat
from langchain_ollama import OllamaEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import ChatOllama
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_core.output_parsers import StrOutputParser
import os
from langchain_chroma import Chroma
import chromadb

In [2]:
import os
os.chdir("..")

In [ ]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_ollama import OllamaEmbeddings
import chromadb
from langchain_chroma import Chroma

# # 📌 Schritt 1: PDFs laden
# pdf_dir = "PDF_docs/"
# pdf_files = [f for f in os.listdir(pdf_dir) if f.endswith(".pdf")]  # Nur PDFs

# all_docs = []
# for pdf_file in pdf_files:
#     pdf_path = os.path.join(pdf_dir, pdf_file)
#     loader = PyPDFLoader(pdf_path)
#     docs = loader.load()
#     all_docs.extend(docs)  # Dokumente speichern

# print(f"✅ {len(all_docs)} Dokumente geladen.")

# # 📌 Schritt 2: Embeddings erstellen
# embeddings = OllamaEmbeddings(model="llama3.2:latest")  # Korrekte Embeddings

# # 📌 Schritt 3: ChromaDB persistent speichern
# persistent_client = chromadb.PersistentClient(path="./chroma_langchain_db")  # Verzeichnis für Speicherung
# collection = persistent_client.get_or_create_collection("collection_name")

# 📌 Schritt 4: Dokumente zu ChromaDB hinzufügen
doc_texts = [doc.page_content for doc in all_docs]  # Extrahiere den Text
doc_ids = [f"doc_{i}" for i in range(len(doc_texts))]  # Einzigartige IDs

collection.add(ids=doc_ids, documents=doc_texts)  # Speichern in ChromaDB

# 📌 Schritt 5: Chroma-VectorStore mit gespeicherten Daten initialisieren
vector_store_from_client = Chroma(
    persist_directory="./chroma_langchain_db",
    collection_name="collection_name",
    embedding_function=embeddings,
)

print("✅ ChromaDB erfolgreich mit PDFs initialisiert!")


In [3]:
file_path = "PDF_docs/"

def save_doc(file_path):
    # Schritt 1: PDFs laden
    file_path = "PDF_docs/"
    pdf_files = [f for f in os.listdir(file_path) if f.endswith(".pdf")]  # Nur PDFs

    all_docs = []
    for pdf_file in pdf_files:
        pdf_path = os.path.join(file_path, pdf_file)
        loader = PyPDFLoader(pdf_path)
        docs = loader.load()
        all_docs.extend(docs)  # Dokumente speichern
    
    return all_docs

In [4]:
all_docs = save_doc(file_path=file_path)
len(all_docs)

117

In [5]:
def embedding():
    embeddings = OllamaEmbeddings(model="granite-embedding:30m")  # Korrekte Embeddings
    return embeddings

In [6]:
embeddings = embedding() 
embeddings

OllamaEmbeddings(model='granite-embedding:30m', base_url=None, client_kwargs={})

In [7]:
def persistent_clientChroma():
    persistent_client = chromadb.PersistentClient(path="./chroma_langchain_db")  # Verzeichnis für Speicherung
    collection = persistent_client.get_or_create_collection("collection_name")
    return collection

collection = persistent_clientChroma()
collection

Collection(name=collection_name)

In [8]:
def add_doc_to_Chroma():
    doc_texts = [doc.page_content for doc in all_docs]  # Extrahiere den Text
    doc_ids = [f"doc_{i}" for i in range(len(doc_texts))]  # Einzigartige IDs

    collection_db = collection.add(ids=doc_ids, documents=doc_texts)  # Speichern in ChromaDB
    return collection_db

In [9]:
collection_db = add_doc_to_Chroma()
collection_db

In [10]:
def vector_store_from_clientChroma():
    # 📌 Schritt 5: Chroma-VectorStore mit gespeicherten Daten initialisieren
    vector_store_from_client = Chroma(
        persist_directory="./chroma_langchain_db",
        collection_name="collection_name",
        embedding_function=embeddings,
    )
    return vector_store_from_client

In [11]:
vector_store_from_client= vector_store_from_clientChroma()
vector_store_from_client

In [12]:
results = vector_store_from_client.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

In [14]:
print(len(embeddings.embed_query("test")))  # Gibt die Anzahl der Dimensionswerte aus


384


In [ ]:
import shutil
shutil.rmtree("./chroma_langchain_db")  # Achtung: Löscht alle gespeicherten Daten
